# Pandas Cookbook 🐼

My Pandas skills are weak. Going through the [Cookbook](https://pandas.pydata.org/pandas-docs/stable/cookbook.html) hopefully helps. An evening chore on 2017-07-21.

![](https://s-media-cache-ak0.pinimg.com/236x/fb/ce/f7/fbcef77e8b855e9bbb5c70fc7991ca2f--cooking-panda-panda-food.jpg)

In [144]:
import pandas as pd
import numpy as np
import functools
import itertools

## Idioms

### if-then, and if-then-else stuff

Conditions with if-else style

In [2]:
df = pd.DataFrame(
    {'AAA': [4, 5, 6, 7],
     'BBB': [10, 20, 30, 40],
     'CCC': [100, 50, -30, -50]}); df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [3]:
df.loc[df.AAA >= 5, 'BBB'] = -1; df

,AAA,BBB,CCC
0,4,10,100
1,5,-1,50
2,6,-1,-30
3,7,-1,-50


In [5]:
df.loc[df.AAA >= 5, ['BBB', 'CCC']] = 555; df

,AAA,BBB,CCC
0,4,10,100
1,5,555,555
2,6,555,555
3,7,555,555


In [7]:
df.loc[df.AAA < 5, ['BBB', 'CCC']] = 2000; df

,AAA,BBB,CCC
0,4,2000,2000
1,5,555,555
2,6,555,555
3,7,555,555


In [10]:
df_mask = pd.DataFrame({'AAA': [True] * 4,
                        'BBB': [False] * 4,
                        'CCC': [True, False] * 2})
df.where(df_mask, -1000)

,AAA,BBB,CCC
0,4,-1000,2000
1,5,-1000,-1000
2,6,-1000,555
3,7,-1000,-1000


In [12]:
df = pd.DataFrame(
    {'AAA': [4, 5, 6, 7],
     'BBB': [10, 20, 30, 40],
     'CCC': [100, 50, -30, 50]}); df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,50


Using `np.where`, which is not the same as Pandas' own `DataFrame.where`.

In [15]:
df['logic'] = np.where(df['AAA'] > 5, 'high', 'low'); df

,AAA,BBB,CCC,logic
0,4,10,100,low
1,5,20,50,low
2,6,30,-30,high
3,7,40,50,high


### Splitting

With boolean criterion

In [17]:
df = pd.DataFrame({
    'AAA': [4, 5, 6, 7],
    'BBB': [10, 20, 30, 40],
    'CCC': [100, 50, -30, -50]}); df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [18]:
dflow = df[df.AAA <= 5]; dflow

,AAA,BBB,CCC
0,4,10,100
1,5,20,50


In [19]:
dfhigh = df[df.AAA > 5]; dfhigh

,AAA,BBB,CCC
2,6,30,-30
3,7,40,-50


### Building criteria

In [20]:
newseries = df.loc[(df['BBB'] < 25) & (df['CCC'] >= -40), 'AAA']; newseries

0    4
1    5
Name: AAA, dtype: int64

In [23]:
newseries = df.loc[(df['BBB'] > 25) | (df['CCC'] >= -40), 'AAA']; newseries

0    4
1    5
2    6
3    7
Name: AAA, dtype: int64

In [24]:
df.loc[(df['BBB'] > 25) | (df['CCC'] >= 75), 'AAA'] = 0.1; df

,AAA,BBB,CCC
0,0.1,10,100
1,5.0,20,50
2,0.1,30,-30
3,0.1,40,-50


Note the use of `DataFrame.loc` sometimes, and sometimes not. A statement like `df['BBB'] > 20` return a Boolean series, which is then used as argument for slicing with `.loc`.

In [31]:
df = pd.DataFrame({
    'AAA': [4, 5, 6, 7],
    'BBB': [10, 20, 30, 40],
    'CCC': [100, 50, -30, -50]
}); df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [36]:
aValue = 43.0
df.loc[(df.CCC - aValue).abs().argsort()]

,AAA,BBB,CCC
1,5,20,50
0,4,10,100
2,6,30,-30
3,7,40,-50


In [38]:
Crit1 = df.AAA <= 5.5
Crit2 = df.BBB == 10.0
Crit3 = df.CCC > -40.0
Crit1, Crit2, Crit3

(0     True
 1     True
 2    False
 3    False
 Name: AAA, dtype: bool, 0     True
 1    False
 2    False
 3    False
 Name: BBB, dtype: bool, 0     True
 1     True
 2     True
 3    False
 Name: CCC, dtype: bool)

In [40]:
AllCrit = Crit1 & Crit2 & Crit2; AllCrit

0     True
1    False
2    False
3    False
dtype: bool

In [43]:
CritList = [Crit1, Crit2, Crit3]
AllCrit = functools.reduce(lambda x,y: x & y, CritList)
AllCrit

0     True
1    False
2    False
3    False
dtype: bool

In [44]:
df[AllCrit]

,AAA,BBB,CCC
0,4,10,100


## Selection

### DataFrames

In [46]:
df = pd.DataFrame({
    'AAA': [4, 5, 6, 7],
    'BBB': [10, 20, 30, 40],
    'CCC': [100, 50, -30, -50]}) ;df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [47]:
df[(df.AAA <= 6) & (df.index.isin([0, 2, 4]))]

,AAA,BBB,CCC
0,4,10,100
2,6,30,-30


In [48]:
data = {'AAA': [4, 5, 6, 7], 'BBB': [10, 20, 30, 40], 'CCC': [100, 50, -30, -50]}
df = pd.DataFrame(data=data, index=['foo', 'bar', 'boo', 'kar']); df

,AAA,BBB,CCC
foo,4,10,100
bar,5,20,50
boo,6,30,-30
kar,7,40,-50


There are 2 explicit slicing methods, with a third general case:
1. Position-oriented (exclusive of end)
1. Label-oriented (inclusive of end)
3. General (depends whetehr slice contains labels or positions)

In [49]:
df.loc['bar':'kar'] # label-oriented

,AAA,BBB,CCC
bar,5,20,50
boo,6,30,-30
kar,7,40,-50


In [50]:
df.iloc[0:3] #generic

,AAA,BBB,CCC
foo,4,10,100
bar,5,20,50
boo,6,30,-30


In [52]:
df2 = pd.DataFrame(data=data, index=[1,2,3,4]) # Note index starts at 1
df2.iloc[1:3] # Position oriented, note the *i* in *iloc*

,AAA,BBB,CCC
2,5,20,50
3,6,30,-30


In [53]:
df2.loc[1:3] # Label-oriented

,AAA,BBB,CCC
1,4,10,100
2,5,20,50
3,6,30,-30


The inverse operator ~ to take the complement of a mask

In [58]:
df = pd.DataFrame(data=data)
df[~((df.AAA <= 6) & (df.index.isin([0, 2, 4])))]

,AAA,BBB,CCC
1,5,20,50
3,7,40,-50


### Panels

Extend a panel frame by transposing, adding a new dimension, and transposing back to the original dimensions.

Panel is a higher level thing. Informally vector (`pd.Series`) < matrix (`pd.DataFrame`) < tensor (`pd.Panel`).

In [72]:
rng = pd.date_range('1/1/2013', periods=100, freq='D') # nice
data = np.random.randn(100, 4)
cols = ['A', 'B', 'C', 'D']
df1, df2, df3 = pd.DataFrame(data, rng, cols), pd.DataFrame(data, rng, cols), pd.DataFrame(data, rng, cols)
df1.head()

,A,B,C,D
2013-01-01,0.279060,-0.368780,-1.123987,0.910866
2013-01-02,0.376830,0.166363,-2.059774,0.082667
2013-01-03,-1.164531,1.170339,0.085776,-0.499174
2013-01-04,-1.845908,0.439836,0.353549,1.112872
2013-01-05,0.641863,2.032675,-1.143256,-0.102743


In [73]:
pf = pd.Panel({'df1': df1, 'df2': df2, 'df3': df3}); pf

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 100 (major_axis) x 4 (minor_axis)
Items axis: df1 to df3
Major_axis axis: 2013-01-01 00:00:00 to 2013-04-10 00:00:00
Minor_axis axis: A to D

In [74]:
pf = pf.transpose(2, 0, 1); pf

<class 'pandas.core.panel.Panel'>
Dimensions: 4 (items) x 3 (major_axis) x 100 (minor_axis)
Items axis: A to D
Major_axis axis: df1 to df3
Minor_axis axis: 2013-01-01 00:00:00 to 2013-04-10 00:00:00

In [75]:
pf['E'] = pd.DataFrame(data, rng, cols)

In [76]:
pf = pf.transpose(1,2,0); pf

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 100 (major_axis) x 5 (minor_axis)
Items axis: df1 to df3
Major_axis axis: 2013-01-01 00:00:00 to 2013-04-10 00:00:00
Minor_axis axis: A to E

In [77]:
pf.loc[:,:,'F'] = pd.DataFrame(data,rng,cols); pf

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 100 (major_axis) x 6 (minor_axis)
Items axis: df1 to df3
Major_axis axis: 2013-01-01 00:00:00 to 2013-04-10 00:00:00
Minor_axis axis: A to F

### New Columns

In [78]:
df = pd.DataFrame({'AAA': [1,2,1,3], 'BBB': [1,1,2,2], 'CCC': [2,1,3,1]}); df

,AAA,BBB,CCC
0,1,1,2
1,2,1,1
2,1,2,3
3,3,2,1


In [79]:
source_cols = df.columns
new_cols = [str(x) + "_cat" for x in source_cols]
categories = {1: 'Alpha', 2: 'Beta', 3: 'Charlie'}
df[new_cols] = df[source_cols].applymap(categories.get); df

,AAA,BBB,CCC,AAA_cat,BBB_cat,CCC_cat
0,1,1,2,Alpha,Alpha,Beta
1,2,1,1,Beta,Alpha,Alpha
2,1,2,3,Alpha,Beta,Charlie
3,3,2,1,Charlie,Beta,Alpha


Keep other columns when using `min()` with `groupby`.

In [80]:
df = pd.DataFrame({'AAA': [1,1,1,2,2,2,3,3], 'BBB': [2,1,3,4,5,1,2,3]}); df

,AAA,BBB
0,1,2
1,1,1
2,1,3
3,2,4
4,2,5
5,2,1
6,3,2
7,3,3


Method 1, use `.idxmin()` to get the index of the mins

In [81]:
df.loc[df.groupby("AAA")["BBB"].idxmin()]

,AAA,BBB
1,1,1
5,2,1
6,3,2


Method 2, sort and then take first of each. This does not return the index label

In [87]:
df.sort_values(by="BBB").groupby("AAA", as_index=False).first()

,AAA,BBB
0,1,1
1,2,1
2,3,2


## MultiIndexing

I should not be learning this now.

In [120]:
df = pd.DataFrame({'row': [0,1,2],
                  'One_X': [1.1, 1.1, 1.1],
                  'One_Y': [1.2, 1.2, 1.2],
                  'Two_X': [1.11, 1.11, 1.11],
                  'Two_Y': [1.22, 1.22, 1.22]}) ;df

,One_X,One_Y,Two_X,Two_Y,row
0,1.1,1.2,1.11,1.22,0
1,1.1,1.2,1.11,1.22,1
2,1.1,1.2,1.11,1.22,2


In [121]:
df = df.set_index('row'); df

,One_X,One_Y,Two_X,Two_Y
row,,,,
0,1.1,1.2,1.11,1.22
1,1.1,1.2,1.11,1.22
2,1.1,1.2,1.11,1.22


View hierarchical columns

In [122]:
df.columns = pd.MultiIndex.from_tuples([tuple(c.split('_')) for c in df.columns]);df

One        Two      
       X    Y     X     Y
row                      
0    1.1  1.2  1.11  1.22
1    1.1  1.2  1.11  1.22
2    1.1  1.2  1.11  1.22

In [123]:
df = df.stack(0).reset_index(1); df

,level_1,X,Y
row,,,
0,One,1.10,1.20
0,Two,1.11,1.22
1,One,1.10,1.20
1,Two,1.11,1.22
2,One,1.10,1.20
2,Two,1.11,1.22


In [124]:
df.columns = ['Sample', 'All_X', 'All_Y']; df

,Sample,All_X,All_Y
row,,,
0,One,1.10,1.20
0,Two,1.11,1.22
1,One,1.10,1.20
1,Two,1.11,1.22
2,One,1.10,1.20
2,Two,1.11,1.22


### Arithmetic

In [126]:
cols = pd.MultiIndex.from_tuples([(x,y) for x in ['A', 'B', 'C'] for y in ['O', 'I']])
df = pd.DataFrame(np.random.randn(2, 6), index=['n', 'm'], columns=cols); df

A                   B                   C          
          O         I         O         I         O         I
n -0.622762 -1.786779 -2.660241  1.567434  0.766322  2.053865
m -1.082098  0.025815 -1.443542 -0.444258 -1.786695 -0.086227

Just to get that list comprehension bit straight

In [127]:
[(x,y) for x in ['A', 'B', 'C'] for y in ['O', 'I']]

[('A', 'O'), ('A', 'I'), ('B', 'O'), ('B', 'I'), ('C', 'O'), ('C', 'I')]

`pd.DataFrame.div()` is an elementwise divide.

In [138]:
df = df.div(df['C'],level=1); df.div


<bound method _arith_method_FRAME.<locals>.f of           A                  B              C     
          O         I        O         I    O    I
n -0.812664 -0.869959 -3.47144  0.763163  1.0  1.0
m  0.605642 -0.299381  0.80794  5.152206  1.0  1.0>

Nope I should not be learning this, more basics please.

### Slicing

In [141]:
coords = [('AA', 'one'), ('AA', 'six'), ('BB', 'one'), ('BB', 'two'), ('BB', 'six')]
index = pd.MultiIndex.from_tuples(coords)
df = pd.DataFrame([11,22,33,44,55], index, ['MyData']); df

MyData
AA one      11
   six      22
BB one      33
   two      44
   six      55

`pd.DataFrame.xs()` returns a cross-section.

In [142]:
df.xs('BB', level=0, axis=0) # level and axis are options, and default to  zero

,MyData
one,33
two,44
six,55


...and now the 2nd level of the 1st axis.

In [143]:
df.xs('six', level=1, axis=0)

,MyData
AA,22
BB,55


Slicing a multi-index with xs, second method

In [151]:
index = list(itertools.product(['Ada', 'Quinn', 'Violet'], ['Comp', 'Math', 'Sci']))
headr = list(itertools.product(['Exams', 'Labs'], ['I', 'II']))
index, headr

([('Ada', 'Comp'),
  ('Ada', 'Math'),
  ('Ada', 'Sci'),
  ('Quinn', 'Comp'),
  ('Quinn', 'Math'),
  ('Quinn', 'Sci'),
  ('Violet', 'Comp'),
  ('Violet', 'Math'),
  ('Violet', 'Sci')],
 [('Exams', 'I'), ('Exams', 'II'), ('Labs', 'I'), ('Labs', 'II')])

In [152]:
indx = pd.MultiIndex.from_tuples(index, names=['Student', 'Course'])
cols = pd.MultiIndex.from_tuples(headr)
data = [[70+x+y+(x*y)%3 for x in range(4)] for y in range(9)]
df = pd.DataFrame(data, indx, cols); df

Exams     Labs    
                   I  II    I  II
Student Course                   
Ada     Comp      70  71   72  73
        Math      71  73   75  74
        Sci       72  75   75  75
Quinn   Comp      73  74   75  76
        Math      74  76   78  77
        Sci       75  78   78  78
Violet  Comp      76  77   78  79
        Math      77  79   81  80
        Sci       78  81   81  81

What the hell is `slice`?

In [156]:
All = slice(None); All

slice(None, None, None)

In [153]:
df.loc['Violet']

Exams     Labs    
           I  II    I  II
Course                   
Comp      76  77   78  79
Math      77  79   81  80
Sci       78  81   81  81

In [155]:
df.loc[(All, 'Math'), All]

Exams     Labs    
                   I  II    I  II
Student Course                   
Ada     Math      71  73   75  74
Quinn   Math      74  76   78  77
Violet  Math      77  79   81  80

In [158]:
df.loc[(slice('Ada', 'Quinn'), 'Math'), All]

Exams     Labs    
                   I  II    I  II
Student Course                   
Ada     Math      71  73   75  74
Quinn   Math      74  76   78  77

In [159]:
df.loc[(All, 'Math'), ('Exams')]

,,I,II
Student,Course,,
Ada,Math,71,73
Quinn,Math,74,76
Violet,Math,77,79


In [160]:
df.loc[(All, 'Math'), (All, 'II')]

,,Exams,Labs
,,II,II
Student,Course,,
Ada,Math,73,74
Quinn,Math,76,77
Violet,Math,79,80


### Sorting

In [162]:
df.sort_values(by=('Labs', 'II'), ascending=False)

Exams     Labs    
                   I  II    I  II
Student Course                   
Violet  Sci       78  81   81  81
        Math      77  79   81  80
        Comp      76  77   78  79
Quinn   Sci       75  78   78  78
        Math      74  76   78  77
        Comp      73  74   75  76
Ada     Sci       72  75   75  75
        Math      71  73   75  74
        Comp      70  71   72  73

## Missing data

This examples fills forwards a reversed timeseries.

In [166]:
df = pd.DataFrame(np.random.randn(6,1), index=pd.date_range('2013-08-01', periods=6, freq='B'), columns=list('A'))
df.loc[df.index[3], 'A'] = np.nan
df

,A
2013-08-01,-0.630452
2013-08-02,1.860921
2013-08-05,-1.755912
2013-08-06,NaN
2013-08-07,-1.174861
2013-08-08,-0.673754


In [171]:
df.reindex(df.index[::-1]).ffill()

,A
2013-08-08,-0.673754
2013-08-07,-1.174861
2013-08-06,-1.174861
2013-08-05,-1.755912
2013-08-02,1.860921
2013-08-01,-0.630452


Yes there is `.bfill()` too, to fill backwards.

## Replacing

Refers to replacing docs.